<a href="https://colab.research.google.com/github/sharzoom/Angular-8-SpringBoot-CRUD-Project/blob/master/Cervical_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [5]:
import tensorflow as tf
import cv2

from sklearn.preprocessing import LabelEncoder
from keras.applications.resnet import ResNet50
from keras.applications.densenet import DenseNet121
from keras.applications.efficientnet import EfficientNetB5
from keras.applications import imagenet_utils
import numpy as np
import pickle
import os

import matplotlib.pyplot as plt
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [6]:
data=[]
labels=[]

height = 64
width = 64
channels = 3
classes = 43
n_inputs = height * width * channels

path1 = "../input/cervical-cancer-largest-dataset-sipakmed/im_Dyskeratotic/im_Dyskeratotic/CROPPED/"
path2 = "../input/cervical-cancer-largest-dataset-sipakmed/im_Koilocytotic/im_Koilocytotic/CROPPED/"
path3 = "../input/cervical-cancer-largest-dataset-sipakmed/im_Metaplastic/im_Metaplastic/CROPPED/"
path4 = "../input/cervical-cancer-largest-dataset-sipakmed/im_Parabasal/im_Parabasal/CROPPED/"
path5 = "../input/cervical-cancer-largest-dataset-sipakmed/im_Superficial-Intermediate/im_Superficial-Intermediate/CROPPED/"

path = [path1, path2, path3, path4, path5]


# New section

In [9]:
# print(Class)
i = 0
for p in path:
    Class = os.listdir(p)
    for a in Class:
        if(a[-1] == 'p'):
            labels.append(i)
            try:
                image=cv2.imread(p+a)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((height, width))
                data.append(np.array(size_image))
            except AttributeError:
                print(" ")
    i+=1

# Converting data to ndarray -
labels = np.array(labels)
data = np.array(data)

print(data.shape)
print(labels.shape)
print(labels)

# Randomize the order of the input images
s = np.arange(data.shape[0])
np.random.seed(43)
np.random.shuffle(s)
data = data[s]
labels = labels[s]

FileNotFoundError: [Errno 2] No such file or directory: '../input/cervical-cancer-largest-dataset-sipakmed/im_Dyskeratotic/im_Dyskeratotic/CROPPED/'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms

# Define your Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self, input_channels=3, num_classes=5):
        super(VisionTransformer, self).__init__()

        # Vision Transformer layers (simplified for illustration)
        self.embedding = nn.Linear(input_channels * 256 * 256, 512)  # Adjust input size as needed
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers=4)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # Flatten input for the linear layer
        x = x.view(x.size(0), -1)

        # Transformer Encoder
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add sequence length dimension (1 in this case)
        x = self.transformer(x)

        # Global average pooling
        x = x.mean(dim=1)

        # Fully connected layer for classification
        x = self.fc(x)

        return x

In [2]:
# Set your device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Load the dataset
dataset = datasets.ImageFolder(root='/kaggle/input/cervical-cancer-largest-dataset-sipakmed/', transform=transform)

# Split the dataset into train, val, and test sets
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Set up data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:

# Initialize your Vision Transformer model
model = VisionTransformer().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate as needed

# Train the model (similar to previous example)
num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [ ]:
 # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")


In [ ]:

# Test the model on the test set
model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")